# Part 1: Camera calibration (Intrinsic parameters)

In [1]:
import numpy as np
import cv2
import glob
import os

In [27]:
print(cv2.__version__)

3.3.0


In [2]:
 # termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

In [3]:
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.


In [4]:
images = glob.glob('./Chessboard3/*.JPG')


In [5]:
print(images)

['./Chessboard3/IMG_1068.JPG', './Chessboard3/IMG_1069.JPG', './Chessboard3/IMG_1057.JPG', './Chessboard3/IMG_1061.JPG', './Chessboard3/IMG_1060.JPG', './Chessboard3/IMG_1062.JPG', './Chessboard3/IMG_1063.JPG', './Chessboard3/IMG_1067.JPG', './Chessboard3/IMG_1066.JPG', './Chessboard3/IMG_1072.JPG', './Chessboard3/IMG_1064.JPG', './Chessboard3/IMG_1070.JPG', './Chessboard3/IMG_1059.JPG', './Chessboard3/IMG_1071.JPG', './Chessboard3/IMG_1065.JPG']


In [6]:
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv2.drawChessboardCorners(img, (9,6), corners,ret)
        name = fname.replace("./Chessboard/","")
        cv2.imwrite("./res/part 1/"+str(name),img)
        #print("./res/part 1/"+str(name)+".png")
        #cv2.imshow('img',img)
        #cv2.waitKey(500)
        #cv2.destroyAllWindows()

In [7]:
 ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

In [8]:
print(ret)


1.628830895986753


In [9]:
print(mtx)

[[2.35823484e+03 0.00000000e+00 1.27505685e+03]
 [0.00000000e+00 2.35363433e+03 9.73403504e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [10]:
print(dist)

[[ 0.10990381 -0.01088416 -0.00567028 -0.00663648 -1.30729001]]


In [11]:
[ 0.10990381 -0.01088417 -0.00567028 -0.00663648 -1.30729001]

[-1.22057713]

In [12]:
principalPoint = (mtx[0][2],mtx[1][2])

In [13]:
focal = (mtx[0][0]+mtx[1][1])/2

In [14]:
print(principalPoint)

(1275.0568472391526, 973.4035044444332)


In [15]:
img = cv2.imread('./Chessboard3/IMG_1068.JPG')

In [16]:
dstchess = cv2.undistort(img, mtx, dist)
cv2.imwrite('./calibresult/mytest.png',dstchess)

True

# Part 2: Take the pictures

# Part 3: Compute the relative camera pose

In [20]:
from matplotlib import pyplot as plt

imgo1 = cv2.imread('./Pair image2/myleft.JPG')  #queryimage # left image
imgo2 = cv2.imread('./Pair image2/myright.JPG') #trainimage # right image


## 3.1 undistort the picture

In [21]:
# undistort
dst1 = cv2.undistort(imgo1, mtx, dist)
cv2.imwrite('./calibresult/myleft.png',dst1)
# undistort
dst2 = cv2.undistort(imgo2, mtx, dist)
cv2.imwrite('./calibresult/myright.png',dst1)

True

In [22]:

img1 = cv2.imread('./calibresult/myleft.png')  #queryimage # left image
img1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
img2 = cv2.imread('./calibresult/myright.png') #trainimage # right image
img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

In [23]:
sift = cv2.xfeatures2d.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

# FLANN parameters
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(des1,des2,k=2)

good = []
pts1 = []
pts2 = []

# ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches):
    if m.distance < 0.5*n.distance:
        good.append(m)
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)

In [24]:
print(pts1)

[(1621.437744140625, 17.146039962768555), (1388.9580078125, 18.361845016479492), (1233.2313232421875, 19.114532470703125), (1261.1224365234375, 18.9810733795166), (1917.4710693359375, 43.18162536621094), (43.383846282958984, 130.83721923828125), (289.1320495605469, 157.08511352539062), (177.90304565429688, 200.20382690429688), (289.13873291015625, 200.98934936523438), (290.6659240722656, 210.66624450683594), (384.2173156738281, 211.8491973876953), (2296.642578125, 227.0430145263672), (2296.642578125, 227.0430145263672), (288.4748840332031, 260.0945739746094), (1953.1551513671875, 266.078125), (1953.1551513671875, 266.078125), (1899.458984375, 270.52362060546875), (2212.63427734375, 279.9571533203125), (83.8695068359375, 285.9066162109375), (289.51416015625, 297.6087646484375), (1923.3326416015625, 331.53271484375), (2064.721923828125, 337.5107421875), (2287.54052734375, 343.30963134765625), (2271.723388671875, 350.6832275390625), (2271.723388671875, 350.6832275390625), (280.62771606445

Got the list of best matches from both images

In [25]:
pts1 = np.int32(pts1)
pts2 = np.int32(pts2)
F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.FM_LMEDS)

# We select only inlier points
pts1 = pts1[mask.ravel()==1]
pts2 = pts2[mask.ravel()==1]

In [26]:
print(F)

[[ 4.46112828e-18 -6.16666798e-01  4.50711143e-01]
 [ 6.16666798e-01  4.18793286e-18  4.50711143e-01]
 [-4.50711143e-01 -4.50711143e-01  0.00000000e+00]]


Next we find the epilines. Epilines corresponding to the points in first image is drawn on second image. So mentioning of correct images are important here. We get an array of lines. So we define a new function to draw these lines on the images.

In [ ]:
def drawlines(img1,img2,lines,pts1,pts2):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    r,c = img1.shape
    img1 = cv2.cvtColor(img1,cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2,cv2.COLOR_GRAY2BGR)
    for r,pt1,pt2 in zip(lines,pts1,pts2):
        color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        img1 = cv2.line(img1, (x0,y0), (x1,y1), color,1)
        img1 = cv2.circle(img1,tuple(pt1),5,color,-1)
        img2 = cv2.circle(img2,tuple(pt2),5,color,-1)
    return img1,img2

Now we find the epilines in both the images and draw them.

In [ ]:
# Find epilines corresponding to points in right image (second image) and
# drawing its lines on left image
lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1,1,2), 2,F)
lines1 = lines1.reshape(-1,3)
img5,img6 = drawlines(img1,img2,lines1,pts1,pts2)

# Find epilines corresponding to points in left image (first image) and
# drawing its lines on right image
lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1,1,2), 1,F)
lines2 = lines2.reshape(-1,3)
img3,img4 = drawlines(img2,img1,lines2,pts2,pts1)

plt.subplot(121),plt.imshow(img5)
plt.subplot(122),plt.imshow(img3)
cv2.imwrite("./res/part 2/epilineleft.png", img5)
cv2.imwrite("./res/part 2/epilineright.png", img3)
plt.show()
cv2.destroyAllWindows()

Compute the essential matrix E
https://docs.opencv.org/3.1.0/d9/d0c/group__calib3d.html#ga13f7e34de8fa516a686a56af1196247f

In [ ]:
E, mask = cv2.findEssentialMat(pts1, pts2, focal, principalPoint, method=cv2.RANSAC, prob=0.999, threshold=3.0)

In [ ]:
print(E)

In [ ]:
R1,R2,t = cv2.decomposeEssentialMat(E)

### decompose the essential matrix, we got two rotaion matrixs, R1(3*3) and R2(3*3), and one translation vector (3*1).
### We need the  projection matrix of the first camera and projection matrix of the second camera to calculate the triangulatePoints
### projMatr1 – 3x4 projection matrix of the first camera.
### projMatr2 – 3x4 projection matrix of the second camera.

In [ ]:
print(R1)

In [ ]:
print(R2)

In [ ]:
print(t)

### There are 4 possible combination for the projection matrix.
#### Case 1:  projMatr1 = K [ I 0]   projMatr2 = K[R1 t]
#### Case 2:  projMatr1 = K[ I 0]   projMatr2 = K[R1  -t]
#### Case 3:  projMatr1 = K[ I 0]   projMatr2 = K[R2 t]
#### Case 4:  projMatr1 = K[ I 0]   projMatr2 = K[R2 -t]


In [ ]:
points1 = np.array(pts1).astype(float).T 
points2 = np.array(pts2).astype(float).T

In [ ]:
# make a projection matrix for camera 1
projMatr1 = mtx@np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0]])

In [ ]:
# case 1
projMatr21 = mtx@np.concatenate((R1,t),1)
# case 2
projMatr22 = mtx@np.concatenate((R1,-t),1)
# case 3
projMatr23 = mtx@np.concatenate((R2,t),1)
# case 4
projMatr24 = mtx@np.concatenate((R2,-t),1)

In [ ]:
triangulate1 = cv2.triangulatePoints(projMatr1, projMatr21,points1,points2)
triangulate2 = cv2.triangulatePoints(projMatr1, projMatr22,points1,points2)
triangulate3 = cv2.triangulatePoints(projMatr1, projMatr23,points1,points2)
triangulate4 = cv2.triangulatePoints(projMatr1, projMatr24,points1,points2)

In [ ]:
print(triangulate1.T)

In [ ]:
print(triangulate1[2].min())

#### case 1 got negtive distance, get rid of it.

In [ ]:
print(triangulate2.T)

In [ ]:
print(triangulate2[2].min())

#### case 2 got negtive distance, get rid of it.

In [ ]:
print(triangulate3.T)

In [ ]:
print(triangulate3[2].min())

#### case 3 got negtive distance, so I get rit of this case.

In [ ]:
print(triangulate4.T)

In [ ]:
print(triangulate4[2].min())

#### case 4 has the positive distance , so the (R2,-t) is the right R and t

In [ ]:
print(triangulate4.T)

In [ ]:
triangulate4world = np.true_divide(triangulate4.T[:,:3], triangulate4.T[:,[-1]]).T

In [ ]:
print(triangulate4world.T.min())

In [ ]:
print(triangulate4world.shape)

In [ ]:
pixelh = mtx@triangulate4world

In [ ]:
print(pixelh.T)

In [ ]:
pixel = np.true_divide(pixelh.T[:,:2],pixelh.T[:,[-1]])

In [ ]:
print(points1.T)

In [ ]:
print(pixel)

In [ ]:
print(pixel.shape)

In [ ]:
imgleft = cv2.imread('./Pair image2/myleft.JPG')  # left image
imgright = cv2.imread('./Pair image2/myright.JPG') # right image



In [ ]:
cv2.imshow('left',imgleft)

In [ ]:
for point in pixel.astype(int):
    #print(tuple(point))
    
    imgleft = cv2.circle(imgleft,tuple(point),10,(0, 0, 200),-1)
for point in points1.T.astype(int):
    #print(tuple(point))
    imgleft = cv2.circle(imgleft,tuple(point),5,(200, 0, 0),-1)
cv2.imwrite("./res/part 3/reporject.png", imgleft)
#plt.imshow(imgleft)
#cv2.imshow('left',imgleft )
#cv2.waitKey(0)
#cv2.destroyAllWindows()


# Part 4: Plane-sweeping stereo

### 4.1 compute 20 depth to generate a array of 20 depth

In [ ]:
print(triangulate4world[2])

In [ ]:
dmin = triangulate4world[2].min()

In [ ]:
dmax = triangulate4world[2].max()

In [ ]:
ddepth = (dmax - dmin)/19

In [ ]:
n = [0,0,1]

In [ ]:
print("dmin: ", dmin, "dmax: ", dmax, "ddepth: ", ddepth)

In [ ]:
def darray(dmin, dmax,number):
    da = []
    ddepth = (dmax - dmin)/19
    for i in range(number):
        da.append(dmin+i*ddepth)
    return da

In [ ]:
deptharray = darray(0,dmax,20)

In [ ]:
from numpy.linalg import inv

In [ ]:
def Homo(k,R2,t,n,depth):
    pmatrix1 = np.concatenate((R2,-t), axis=1)
    pmatrix2 = np.append(n,depth).reshape(1,4)
    p = np.concatenate([pmatrix1,pmatrix2])
    H = p@inv(p)
    H = H[0:3,0:3]
    Homo = k@H@inv(k)
    return Homo

In [ ]:
Homo1 = Homo(mtx,R2,t,n,deptharray[18])

In [ ]:
print(Homo1)

In [ ]:
print(deptharray[3])

In [ ]:
for d in deptharray:
    Hh = Homo(mtx,R2,t,n,d)
    warpname = str(d).replace(".","")
    print(warpname)
    print("./res/part 4/warp"+warpname+".png")

In [ ]:
imgr4 = cv2.imread('./Pair image/myright.JPG')
imgrg4 = cv2.cvtColor(imgr4,cv2.COLOR_BGR2GRAY)

In [ ]:
count = 0
for d in deptharray:
    Hh = Homo(mtx,R2,t,n,d)
    imgwarpn = cv2.warpPerspective(imgrg4,Hh,(imgrg4.shape[1],imgrg4.shape[0]))
    count+=1
    cv2.imwrite("./res/part 4/warp"+str(count)+".png", imgwarpn)

In [ ]:
#imgwarpn = cv2.warpPerspective(imgrg4,Homo1,(imgrg4.shape[1],imgrg4.shape[0]))

In [ ]:
#cv2.imwrite("./res/part 4/warp.png", imgwarpn)

### 4.2 compute the depth picture.
#### depth1 = wrap1 - original
#### depth2 = wrap2 - orignial
